<a href="https://colab.research.google.com/github/LennyHenrydoesGitHub/OSINT/blob/main/model/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Packages

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import torch
import torch.nn as nn
from google.colab import files, drive
import pandas as pd
import random
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
drive.mount('/content/drive')

Mounted at /content/drive


### Dictionary for conflicts details

In [2]:
df_conflicts = pd.read_csv('/content/drive/My Drive/Data_sci/icb1v15.csv')
df_actors = pd.read_csv('/content/drive/My Drive/Data_sci/icb2v15.csv')

crisis_details = {}

for index, row in df_conflicts.iterrows():
    crisis_number = row['crisno']  # Crisis number
    crisis_name = row['crisname']  # Crisis name
    start_year = row['yrtrig']  # Start year
    end_year = row['yrterm']  # End year

    crisis_details[crisis_number] = {'crisis_name': crisis_name,
                                      'start_year': start_year,
                                      'end_year': end_year,
                                      'actors': set()}

for index, row in df_actors.iterrows():
    crisis_number = row['crisno']  # Crisis number
    actor = row['actor']  # Actor

    if crisis_number in crisis_details:
        crisis_details[crisis_number]['actors'].add(actor)

### Make matrix for conflicts (i) and actors (j), for if a conflict has been started

In [4]:
actors = set()
for _, details in crisis_details.items():
    actors.update(details['actors'])

matrix_data = []
for _, details in crisis_details.items():
    crisis_year = details['start_year']
    crisis_actors = details['actors']
    row_data = {'Year': crisis_year}
    for actor in actors:
        row_data[actor] = 1 if actor in crisis_actors else 0
    matrix_data.append(row_data)

actor_crisis_matrix = pd.DataFrame(matrix_data)

columns = actor_crisis_matrix.columns.tolist()
columns.remove('Year')
columns = ['Year'] + columns

actor_crisis_matrix = actor_crisis_matrix[columns]
actor_crisis_matrix = actor_crisis_matrix[actor_crisis_matrix['Year'] >= 1950]


### Processing arms trade data

In [21]:
df = pd.read_csv('/content/drive/My Drive/Data_sci/all_alphabetical_by_recipient.csv')
country_codes = pd.read_csv('/content/drive/My Drive/Data_sci/country_codes_15-04.csv')

name_to_code = dict(zip(country_codes['country names'], country_codes['country codes']))
df['Recipient'] = df['Recipient'].map(name_to_code)


### Save needed data

In [22]:
np.save('/content/drive/My Drive/Data_sci/df.npy',df)
np.save('/content/drive/My Drive/Data_sci/matrix.npy',actor_crisis_matrix)